In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM

In [ ]:
train_text_seq_path = "./datasets/train/train_text_seq.csv"
valid_text_seq_path = "./datasets/valid/valid_text_seq.csv"
test_text_seq_path = "./datasets/test/test_text_seq.csv"

In [ ]:
train_df = pd.read_csv(train_text_seq_path)
validation_df = pd.read_csv(valid_text_seq_path)
test_df = pd.read_csv(test_text_seq_path)


# Convert input_str to list of integers (0-9) for each character for training and validation
X_train = np.array([[int(char) for char in seq] for seq in train_df['input_str']])
y_train = np.array(train_df['label'])

X_val = np.array([[int(char) for char in seq] for seq in validation_df['input_str']])
y_val = np.array(validation_df['label'])

# Convert input_str to list of integers for the test set (no labels in the test set)
X_test = np.array([[int(char) for char in seq] for seq in test_df['input_str']])

# Print shapes of the data
print("Training data shape:", X_train.shape)  # Should print (num_samples, 50)
print("Validation data shape:", X_val.shape)  # Should print (num_samples, 50)
print("Test data shape:", X_test.shape)        # Should print (num_samples, 50)

# Model Parameters
max_length = 50  # As mentioned, input strings have a fixed length of 50
vocab_size = 10  # Since digits are between 0-9

def create_gru_model():
    embedding_dim = 64
    model = Sequential()
    model.add(Input(shape=(max_length,)))
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
    model.add(GRU(units=28, return_sequences=False))
    model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


def create_lstm_model():
    embedding_dim = 32
    model = Sequential()
    model.add(Input(shape=(max_length,)))
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
    model.add(LSTM(units=32, return_sequences=False))
    model.add(Dense(38))
#     model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


percentages = np.linspace(0.2, 1.0, 5)
results = []

for percent in percentages:
    num_samples = int(len(X_train) * percent)

    X_train_subset = X_train[:num_samples]
    y_train_subset = y_train[:num_samples]

    print(f"\nTraining with {percent * 100:.1f}% of training data ({num_samples} samples):")

    lstm_model = create_lstm_model()
    gru_model = create_gru_model()

    lstm_model.fit(X_train_subset, y_train_subset, epochs=60, batch_size=16, validation_data=(X_val, y_val),verbose=0)
    lstm_val_loss, lstm_val_acc = lstm_model.evaluate(X_val, y_val)
    print(f"LSTM Validation Accuracy: {lstm_val_acc * 100:.2f}%")
    results.append([percent * 100, "LSTM", lstm_val_acc * 100])

    gru_model.fit(X_train_subset, y_train_subset, epochs=60, batch_size=16, validation_data=(X_val, y_val),verbose=0)
    gru_val_loss, gru_val_acc = gru_model.evaluate(X_val, y_val)
    print(f"GRU Validation Accuracy: {gru_val_acc * 100:.2f}%")
    results.append([percent * 100, "GRU", gru_val_acc * 100])


lstm_model.summary()
gru_model.summary()


df = pd.DataFrame(results, columns=['percent', 'model_name', 'validation_accuracy'])


plt.figure(figsize=(10, 6))

for model_name in df['model_name'].unique():
    model_df = df[df['model_name'] == model_name]
    plt.plot(model_df['percent'], model_df['validation_accuracy'], marker='o', label=model_name)

plt.title('Validation Accuracy vs. Percentage of Data Used')
plt.xlabel('Percentage of Data Used (%)')
plt.ylabel('Validation Accuracy (%)')
plt.legend(title='Model Name')
plt.show()

print(df)